In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil

shutil.copytree('/content/finetuned_aragpt2_sanad', '/content/drive/MyDrive/aragpt2_models/finetuned_aragpt2_sanad')

'/content/drive/MyDrive/aragpt2_models/finetuned_aragpt2_sanad'

In [ ]:
from google.colab import files
files.upload()


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d haithemhermessi/sanad-dataset


!unzip '*.zip'

Streaming output truncated to the last 5000 lines.
  inflating: Tech/1500.txt           
  inflating: Tech/1501.txt           
  inflating: Tech/1502.txt           
  inflating: Tech/1503.txt           
  inflating: Tech/1504.txt           
  inflating: Tech/1505.txt           
  inflating: Tech/1506.txt           
  inflating: Tech/1507.txt           
  inflating: Tech/1508.txt           
  inflating: Tech/1509.txt           
  inflating: Tech/1510.txt           
  inflating: Tech/1511.txt           
  inflating: Tech/1512.txt           
  inflating: Tech/1513.txt           
  inflating: Tech/1514.txt           
  inflating: Tech/1515.txt           
  inflating: Tech/1516.txt           
  inflating: Tech/1517.txt           
  inflating: Tech/1518.txt           
  inflating: Tech/1519.txt           
  inflating: Tech/1520.txt           
  inflating: Tech/1521.txt           
  inflating: Tech/1522.txt           
  inflating: Tech/1523.txt           
  inflating: Tech/1524.txt           

In [ ]:
!pip install datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Importing needed libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import os
import random
import re
from unicodedata import normalize

# Disable wandb

In [ ]:

os.environ["WANDB_DISABLED"] = "true"

# Load tokenizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

# Add pad_token if not present

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


# Verify pad_token

In [ ]:
print(f"Pad token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")

Pad token: [PAD], ID: 64000


# Path to SANAD dataset

In [ ]:
sanad_path = "/content/Data"

# Enhanced preprocessing functions

In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^\u0600-\u06FF\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
def normalize_arabic(text):
    text = re.sub(r'[أإآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = normalize('NFKD', text).replace('\u064B', '').replace('\u064C', '').replace('\u064D', '') \
           .replace('\u064E', '').replace('\u064F', '').replace('\u0650', '').replace('\u0651', '') \
           .replace('\u0652', '')
    return text

In [ ]:
def is_valid_sentence(sentence, min_length=3, max_length=50):
    words = sentence.split()
    if len(words) < min_length or len(words) > max_length:
        return False
    if re.match(r'^[\s.,!?]+$', sentence) or len(set(words)) < 2:
        return False
    return True

# Combine subset for training

In [ ]:
def combine_sanad_subset(sanad_path, output_file="sanad_subset.txt", subset_ratio=0.2, exclude_files=None):
    categories = ["Culture", "Finance", "Medical", "Politics", "Religion", "Sports", "Tech"]
    selected_files = []

    with open(output_file, "w", encoding="utf-8") as outfile:
        for category in categories:
            category_path = os.path.join(sanad_path, category)
            if not os.path.exists(category_path):
                print(f"Warning: {category} folder not found.")
                continue
            files = [f for f in os.listdir(category_path) if f not in (exclude_files or [])]
            subset_size = int(len(files) * subset_ratio)
            category_files = random.sample(files, min(subset_size, len(files)))
            selected_files.extend(category_files)

            for filename in category_files:
                with open(os.path.join(category_path, filename), "r", encoding="utf-8") as infile:
                    text = infile.read().strip()
                    text = clean_text(text)
                    text = normalize_arabic(text)
                    sentences = text.split(". ")
                    for sentence in sentences:
                        sentence = sentence.strip()
                        if sentence and is_valid_sentence(sentence):
                            outfile.write(sentence + "\n")
    return selected_files

# Combine test subset

In [ ]:
def combine_test_subset(sanad_path, output_file="sanad_test.txt", train_files=None):
    categories = ["Culture", "Finance", "Medical", "Politics", "Religion", "Sports", "Tech"]
    test_pairs = []

    with open(output_file, "w", encoding="utf-8") as outfile:
        for category in categories:
            category_path = os.path.join(sanad_path, category)
            if not os.path.exists(category_path):
                print(f"Warning: {category} folder not found.")
                continue
            files = [f for f in os.listdir(category_path) if f not in (train_files or [])]
            subset_size = min(int(len(files) * 0.05), len(files))
            category_files = random.sample(files, min(subset_size, len(files)))

            for filename in category_files:
                with open(os.path.join(category_path, filename), "r", encoding="utf-8") as infile:
                    text = infile.read().strip()
                    text = clean_text(text)
                    text = normalize_arabic(text)
                    sentences = text.split(". ")
                    for sentence in sentences:
                        sentence = sentence.strip()
                        if sentence and is_valid_sentence(sentence):
                            words = sentence.split()
                            if len(words) > 1:
                                prompt = " ".join(words[:-1])
                                target = words[-1]
                                test_pairs.append((prompt, target))
                                outfile.write(sentence + "\n")
    return test_pairs

# Combine training subset

In [ ]:
train_files = combine_sanad_subset(sanad_path, subset_ratio=0.2)

# Combine test subset (excluding train files)

In [ ]:
test_pairs = combine_test_subset(sanad_path, train_files=train_files)

# Load training dataset

In [ ]:
dataset = load_dataset("text", data_files={"train": "sanad_subset.txt"})

Generating train split: 0 examples [00:00, ? examples/s]

# Tokenize training dataset

In [ ]:
def tokenize_function(examples):
    encodings = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)
    encodings["labels"] = encodings["input_ids"].copy()
    return encodings


In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/46800 [00:00<?, ? examples/s]

# Debug: Check dataset

In [ ]:
print("Dataset features:", tokenized_dataset["train"].features)
print("Sample data:", tokenized_dataset["train"][0])

Dataset features: {'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
Sample data: {'input_ids': [330, 4884, 2917, 3071, 35820, 3275, 530, 1712, 1022, 1457, 308, 4870, 12385, 4204, 1046, 9403, 152, 238, 2928, 1046, 1589, 410, 1940, 46315, 3150, 1303, 4870, 1593, 38169, 4222, 152, 238, 1212, 1028, 1175, 19184, 10360, 308, 802, 20571, 2929, 22891, 509, 1731, 13144, 17, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000, 64000], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [330, 4884, 2917, 3071, 35820, 3275, 530, 1712, 1022, 1457, 308, 4870, 12385, 4204, 1

# Set up training arguments with checkpointing

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/finetuned_aragpt2_sanad",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    save_steps=1000,
    save_total_limit=3,
    logging_dir="/content/logs",
    logging_steps=200,
    fp16=True,
    save_strategy="steps",
    learning_rate=2e-5,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Custom Trainer to debug inputs

In [ ]:
class DebugTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        print("Inputs to model:", inputs.keys())
        return super().compute_loss(model, inputs, return_outputs)

# Initialize Trainer

In [ ]:
trainer = DebugTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Check for checkpoints and fine-tune

In [ ]:
checkpoint_dir = "/content/finetuned_aragpt2_sanad"
resume = os.path.exists(checkpoint_dir) and any("checkpoint" in d for d in os.listdir(checkpoint_dir))
trainer.train(resume_from_checkpoint=resume)

Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])


Step,Training Loss
200,39.749900
400,18.056500
600,9.141100
800,8.604400
1000,8.507600
1200,8.165100
1400,8.350900
1600,7.934000
1800,8.057300
2000,7.935800


Streaming output truncated to the last 5000 lines.
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids', 'attention_mask', 'labels'])
Inputs to model: dict_keys(['input_ids'

TrainOutput(global_step=11700, training_loss=7.99699454022269, metrics={'train_runtime': 2832.3405, 'train_samples_per_second': 33.047, 'train_steps_per_second': 4.131, 'total_flos': 3057116774400000.0, 'train_loss': 7.99699454022269, 'epoch': 2.0})

# Save final model

In [ ]:
model.save_pretrained("/content/finetuned_aragpt2_sanad")
tokenizer.save_pretrained("/content/finetuned_aragpt2_sanad")


('/content/finetuned_aragpt2_sanad/tokenizer_config.json',
 '/content/finetuned_aragpt2_sanad/special_tokens_map.json',
 '/content/finetuned_aragpt2_sanad/vocab.json',
 '/content/finetuned_aragpt2_sanad/merges.txt',
 '/content/finetuned_aragpt2_sanad/added_tokens.json',
 '/content/finetuned_aragpt2_sanad/tokenizer.json')

# Evaluate accuracy

In [ ]:
def predict_top_k_next_words(prompt, top_k=1):
    if not prompt.strip():
        return []
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        outputs = model(input_ids)
    logits = outputs.logits[:, -1, :]
    probs = torch.softmax(logits, dim=-1)
    top_k_probs, top_k_ids = torch.topk(probs, top_k)
    predictions = [tokenizer.decode([idx]).strip() for idx in top_k_ids[0]]
    return predictions

# Move model to GPU if available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

correct = 0
total = 0
for prompt, target in test_pairs:
    predictions = predict_top_k_next_words(prompt, top_k=1)
    if predictions and predictions[0] == target:
        correct += 1
    total += 1

accuracy = (correct / total) * 100 if total > 0 else 0
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total} correct predictions)")

Accuracy: 25.53% (679/2660 correct predictions)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/finetuned_aragpt2_sanad"
if not os.path.exists(model_path):
    print(f"Error: Model directory '{model_path}' does not exist.")
    exit()

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def predict_top_k_next_words(prompt, top_k=3):
    try:
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
        with torch.no_grad():
            outputs = model(input_ids)
        logits = outputs.logits[:, -1, :]
        probs = torch.softmax(logits, dim=-1)
        top_k_probs, top_k_ids = torch.topk(probs, top_k)
        predictions = [tokenizer.decode([idx]).strip() for idx in top_k_ids[0]]
        return predictions
    except Exception as e:
        print(f"Error predicting for prompt '{prompt}': {e}")
        return []

prompts = [
    "الفن العربي في",
    "السوق المالي في",
    "العلاج الجديد لمرض",
    "الانتخابات في",
    "الشعائر الدينية في",
    "مباراة كرة القدم بين",
    "التكنولوجيا الحديثة تشمل"
]

for prompt in prompts:
    suggestions = predict_top_k_next_words(prompt, top_k=3)
    print(f"Prompt: {prompt}")
    print("Next word suggestions:", suggestions)
    print()

Model loaded successfully!
Prompt: الفن العربي في
Next word suggestions: ['الامارات', 'عام', 'معرض']

Prompt: السوق المالي في
Next word suggestions: ['الامارات', 'ابوظبي', 'دبي']

Prompt: العلاج الجديد لمرض
Next word suggestions: ['السكري', 'السرطان', 'السكر']

Prompt: الانتخابات في
Next word suggestions: ['مصر', 'موعدها', 'العراق']

Prompt: الشعائر الدينية في
Next word suggestions: ['الاسلام', 'مصر', 'الامارات']

Prompt: مباراة كرة القدم بين
Next word suggestions: ['البرازيل', 'الفريقين', 'فريقي']

Prompt: التكنولوجيا الحديثة تشمل
Next word suggestions: ['اجهزة', 'تطبيقات', 'تقنية']



In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import defaultdict

def evaluate_auto_complete(test_pairs, model, tokenizer, top_k=3):

    word_correct = 0
    char_correct = 0
    total_chars = 0
    bleu_scores = []
    top_k_hits = 0
    smoothing = SmoothingFunction().method1

    for prompt, target in test_pairs:

        predictions = predict_top_k_next_words(prompt, top_k=top_k)


        if predictions and predictions[0] == target:
            word_correct += 1


        if target in predictions:
            top_k_hits += 1


        if predictions:
            pred_word = predictions[0]
            min_len = min(len(pred_word), len(target))
            for i in range(min_len):
                if pred_word[i] == target[i]:
                    char_correct += 1
            total_chars += max(len(pred_word), len(target))


        if predictions:
            bleu = sentence_bleu(
                [list(target)],
                list(predictions[0]),
                smoothing_function=smoothing
            )
            bleu_scores.append(bleu)


    results = {
        "word_accuracy": (word_correct / len(test_pairs)) * 100,
        "top_k_accuracy": (top_k_hits / len(test_pairs)) * 100,
        "char_accuracy": (char_correct / total_chars) * 100 if total_chars > 0 else 0,
        "avg_bleu": np.mean(bleu_scores) if bleu_scores else 0,
        "total_test_samples": len(test_pairs)
    }
    return results

# Run evaluation
if __name__ == "__main__":

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)


    metrics = evaluate_auto_complete(test_pairs, model, tokenizer, top_k=3)


    print("\n=== Evaluation Results ===")
    print(f"Word-level accuracy (exact match): {metrics['word_accuracy']:.2f}%")
    print(f"Top-3 accuracy (target in suggestions): {metrics['top_k_accuracy']:.2f}%")
    print(f"Character-level accuracy: {metrics['char_accuracy']:.2f}%")
    print(f"Average BLEU score (n-gram similarity): {metrics['avg_bleu']:.4f}")
    print(f"Tested on {metrics['total_test_samples']} samples.")


=== Evaluation Results ===
Word-level accuracy (exact match): 31.24%
Top-3 accuracy (target in suggestions): 44.89%
Character-level accuracy: 38.42%
Average BLEU score (n-gram similarity): 0.3232
Tested on 2660 samples.
